<a href="https://colab.research.google.com/github/Vlasovets/Deep_learning_course_assistantship/blob/master/layout_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced topics in User Interfaces

## **Learning goals** 
The tutorial covers the topics explained during the lecture in the following parts:
1.   to get introduced to the image recognition model and how it can be applied to website layouts.
2.   to get introduced to generators in Python
3.   to train and test models on given datasets
4.   to get introduced to ReLU activation function 

## 1. Introduction

### 1.1. Layout composition 
A website layout is a pattern that defines a website’s structure. It has the role of structuring the information present on a site both for the website’s owner and for users. It provides clear paths for navigation within webpages and puts the most important elements of a website front and center.

There are many reasons to consider a choice of layout wisely, for example:
- A good layout keeps users on the site because it makes important information easily accessible and intuitive to find. A bad layout frustrates users which then quickly leaves the site because they can’t find what they are looking for.

- There’s a strong [relationship](https://blog.hubspot.com/marketing/compelling-stats-website-design-optimization-list) between the layout and the engagement of users with the website. It determines how long they dwell on the website pages, how many pages they browse and how often they come back to the website.

- According to Adobe research, 38% of people will stop engaging with a website if the content/layout is unattractive.

You can think about other reasons why layouts are important. Once you have no doubts, you can start the following exercise.


In [0]:
%tensorflow_version 1.x # this tutorial use TensorFlow v.1

Importing necessary libraries for this tutorial.

If you face the error "No module found:...", make sure that prerequisites from the first exercise session are satisfied and you installed all the packages we use.

*Hint: use the following command to install the package
```
# !pip install #packagename
```



In [2]:
import os
from tensorflow.python.keras.models import Sequential #The Sequential model is a linear stack of layers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

# Some GPUs require setting the `allow_growth` setting.
# Comment out this code is you don't have a GPU card.
import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

In [0]:
!unzip layouts-easy.zip #complementary code for loading the data

## 2. Dataset description
The data for today's exericise session is a number of layouts which can be categorized into two groups:
- <font color=green>good</font> cop (layout)
- <font color=red>bad</font> cop (layout)

The distinction is based on the composition of the layout.


*Please, add some details about the dataset*

### **Bad layout 😭**

In [37]:
from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/IhgEVrZZ2p8g5g3pFh/giphy.gif">')

### **Good Layout😀**

In [45]:
HTML('<img src="https://media.giphy.com/media/lSE2LCTTrae4GEy4Km/giphy.gif">')

The following code assigns the hyperparameters such as the number of epochs and the batch size. The choice of these hyperparameters affects the accuracy and the training time of the model.

In [6]:
IMAGE_SIZE = (200, 200)
BATCH_SIZE = 64
NUM_EPOCHS = 100

dataset_dir = 'layouts-easy'
trainer_dir = dataset_dir + '/train'

# Specify manually the class labels, otherwise Keras will assign alphanumeric values
# as directories are listed (in no particular order).
dirname, class_labels, _ = next(os.walk(trainer_dir))
print('Class labels: {}'.format(class_labels))

Class labels: ['Good', 'Bad']


Data augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data.

Basic augmentation techniques are commonly used to train large neural networks:
- cropping
- padding
- horizontal flipping

More info on data augmentation can be found [here](https://nanonets.com/blog/data-augmentation-how-to-use-deep-learning-when-you-have-limited-data-part-2/).

In [0]:
# Apply some data augmentation techniques.
image_data = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

*Python Note*🤓

In the next cell we use <font color=yellow>*generators*</font> - simple functions which return an iterable set of items, one at a time, in a special way.

When an iteration over a set of item starts using the for statement, the generator is run. Once the generator's function code reaches a "yield" statement, the generator yields its execution back to the for loop, returning a new value from the set. The generator function can generate as many values (possibly infinite) as it wants, yielding each one in its turn.

You can write the following code to understand better the logic generator call uses:


```
import random

def lottery():
    # returns 6 numbers between 1 and 40
    for i in range(6):
        yield random.randint(1, 40)

    # returns a 7th number between 1 and 15
    yield random.randint(1,15)

for random_number in lottery():
       print("And the next number is... %d!" %(random_number))
```




In [8]:
# Use generators since we might not be able to fit all the images into memory.
train_generator = image_data.flow_from_directory(
    trainer_dir,
    subset='training',
    color_mode='grayscale',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', #the model is binary since we have just 2 classes - bad or good
    classes=class_labels
)

Found 320 images belonging to 2 classes.


In [9]:
valid_generator = image_data.flow_from_directory(
    trainer_dir,
    subset='validation',
    color_mode='grayscale',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=class_labels
)


Found 80 images belonging to 2 classes.


## 3. Model training

As you will see we use <font color=orange>ReLU</font> activation function in our model. ReLU stands for *rectified linear unit* Mathematically, it is defined as $f(x_i) = max(0, x_i)$. 

Visually, it looks like the following:

In [46]:
 from IPython.display import Image
 Image(url= "https://miro.medium.com/max/1026/1*DfMRHwxY1gyyDmrIAd-gjQ.png", width=400, height=200)

ReLU is the most commonly used activation function in neural networks, especially in [CNNs](https://www.youtube.com/watch?v=YRhxdVk_sIs). 

If you are unsure what activation function to use in your network, ReLU is usually a good first choice.

In [10]:
# Notice that images are grayscaled, therefore we have to set the number of channels to 1.
# Image shape is `(width, height, num_channels)` in TensorFlow.
in_shape = IMAGE_SIZE + (1,)

model = Sequential()
model.add(Conv2D(3, (3,3), activation='relu', input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 3)       30        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 3)         0         
_________________________________________________________________
dropout (Dropout)            (None, 99, 99, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 29403)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 29404     
Total params: 29,434
Trainable params: 29,434
Non-trainable params: 0
_________

In [0]:
# Setup some useful callbacks.
visualizer = TensorBoard(log_dir='/tmp/layouts')
checkpoint = ModelCheckpoint('{}-best.h5'.format(dataset_dir), monitor='val_acc', mode='max', save_best_only=True)
earlystops = EarlyStopping(patience=10)

In [12]:
# Train the model.
model.fit_generator(
    train_generator,
    epochs=NUM_EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=[visualizer, checkpoint, earlystops]
)

Epoch 1/100
5/5 [==============================] - 5s 1s/step - loss: 211.4596 - acc: 0.4688 - val_loss: 321.6348 - val_acc: 0.4688
Epoch 2/100
5/5 [==============================] - 3s 595ms/step - loss: 181.8356 - acc: 0.5344 - val_loss: 82.6008 - val_acc: 0.5312
Epoch 3/100
5/5 [==============================] - 3s 614ms/step - loss: 110.2843 - acc: 0.5000 - val_loss: 47.4562 - val_acc: 0.5312
Epoch 4/100
5/5 [==============================] - 3s 614ms/step - loss: 38.2964 - acc: 0.5281 - val_loss: 50.9043 - val_acc: 0.4688
Epoch 5/100
5/5 [==============================] - 3s 627ms/step - loss: 18.5924 - acc: 0.6531 - val_loss: 12.9848 - val_acc: 0.5625
Epoch 6/100
5/5 [==============================] - 3s 600ms/step - loss: 16.4312 - acc: 0.5781 - val_loss: 21.8360 - val_acc: 0.5625
Epoch 7/100
5/5 [==============================] - 3s 622ms/step - loss: 13.1044 - acc: 0.6250 - val_loss: 6.6347 - val_acc: 0.7969
Epoch 8/100
5/5 [==============================] - 3s 615ms/step - lo

In [13]:
# Report score from last epoch.
loss, acc = model.evaluate_generator(valid_generator)
print('Accuracy: {:.2f}%'.format(acc*100))

# Finally save the model.
model.save('{}.h5'.format(dataset_dir))

Accuracy: 85.00%


As you can see our model showed a very good result reaching the accuracy of 85%. 

Now, we use our model to make predictions on the test data.

## 4. Model Evaluation

In [0]:
import os
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

In [15]:
dataset_dir = 'layouts-easy'
trainer_dir = dataset_dir + '/train'

# Specify manually the class labels, otherwise Keras will assign alphanumeric values
# as directories are listed (in no particular order).
dirname, class_labels, _ = next(os.walk(trainer_dir))
print('Class labels: {}'.format(class_labels))

# Don't compile the model when testing new data.
# model = load_model('{}.h5'.format(dataset_dir), compile=False)

Class labels: ['Good', 'Bad']


In [0]:
model = load_model('{}.h5'.format(dataset_dir), compile=False) #compile the trained model

In [17]:
# Don't shuffle the data, so that we can get the groundtruth labels later.
test_generator = ImageDataGenerator().flow_from_directory(
    dataset_dir + '/test',
    shuffle=False,
    color_mode='grayscale',
    target_size=(200, 200),
    class_mode='binary',
    classes=class_labels
)

Found 100 images belonging to 2 classes.


In [0]:
# Since we have groundtruth labels, we can use `model.evaluate_generator` here.
# However in production we don't have groundtruth so let's use the model to predict new data
# and then compare the predictions against the test labels.
y_true = list(map(lambda f : os.path.dirname(f), test_generator.filenames))
probas = model.predict_generator(test_generator)
y_pred = list(map(lambda p : class_labels[0] if p < 0.5 else class_labels[1], probas))

In [19]:
# Let's see how good those predictions were.
precision, recall, fmeasure, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_labels, average='weighted')
print('Precision: {:.2f}%'.format(precision * 100))
print('Recall: {:.2f}%'.format(recall * 100))
print('F-measure: {:.2f}%'.format(fmeasure * 100))

# Finally compute the ROC AUC to see the discriminative power of the model.
# We'll use the `roc_auc_score()` function, which expects *binary* groundtruth labels and probability estimates of the *positive class*;
# so be careful how labels and probabilities are defined.
binary_labels = [p == y_true[i] for i, p in enumerate(y_pred)]
auc = roc_auc_score(binary_labels, 1 - probas, average='weighted')
print('AUC: {:.2f}%'.format(auc * 100))

Precision: 82.79%
Recall: 79.00%
F-measure: 78.38%
AUC: 64.26%


The final model ended up with accuracy 82.79%, good job!👍

## 5. Conclusion

Now, you know:

1.   how important layout compositions
2.   what is data augmentation
3.   what is a generator function in Python
4.   how to apply a simple sequential model
5.   what is a ReLU activation function

Do not hesitate to ask questions at otorrent@mail.ru

Thank you for your attention and see you next exercise session!